# 3.9 Observabilidad y Monitoreo - Métricas y Alertas en Producción

## 🎯 ¿Para qué?

---

## 🎯 Objetivos de Aprendizaje
- Configurar Query Store para capturar historial de planes de ejecución y métricas de runtime.
- Usar Extended Events (alternativa ligera a SQL Profiler) para auditar queries costosas.
- Diseñar alertas proactivas (umbral de CPU, disco, duración de queries) con SQL Agent.

## 🧰 Prerrequisitos
- Haber completado `08_almacenamiento_y_mantenimiento.ipynb`.
- Conocer DMVs básicas (`sys.dm_exec_requests`, `sys.dm_os_wait_stats`).
- Acceso a SQL Server Agent y permisos VIEW SERVER STATE.

---

## 📚 Conceptos Clave
- **Query Store**: repositorio de planes históricos y estadísticas de ejecución; permite comparar antes/después de cambios.
- **DMVs (Dynamic Management Views)**: vistas del sistema que exponen estado en tiempo real (wait stats, index usage, active queries).
- **Extended Events**: framework de auditoría ligero para capturar eventos (deadlocks, queries >5s, implicit conversions).
- **Baseline**: métricas normales en operación estándar; anomalías se detectan comparando contra baseline.

> ⚠️ Query Store consume espacio; configurar retención (ej. 30 días) y limpieza automática; Extended Events mal filtrados generan overhead.

In [ ]:
-- Query 3: Esperas del servidor (wait stats)
SELECT TOP 10
    wait_type,
    wait_time_ms / 1000.0 AS wait_time_sec,
    waiting_tasks_count AS num_esperas,
    wait_time_ms / waiting_tasks_count AS promedio_ms
FROM sys.dm_os_wait_stats
WHERE wait_type NOT IN (
    'CLR_SEMAPHORE', 'LAZYWRITER_SLEEP', 'RESOURCE_QUEUE', 
    'SLEEP_TASK', 'SLEEP_SYSTEMTASK', 'SQLTRACE_BUFFER_FLUSH', 
    'WAITFOR', 'XE_DISPATCHER_WAIT', 'XE_TIMER_EVENT'
)  -- Filtrar esperas normales del sistema
ORDER BY wait_time_ms DESC;

/*
🔎 Wait types críticos:
- PAGEIOLATCH_*: lecturas/escrituras de disco lentas (I/O bottleneck, agregar RAM o SSD)
- LCK_M_*: locks/bloqueos (optimizar queries, reducir transacciones largas)
- CXPACKET: paralelismo excesivo (ajustar MAXDOP si promedio_ms alto)
- SOS_SCHEDULER_YIELD: CPU saturado (optimizar queries, scale up CPU)
- WRITELOG: escritura de log lenta (mover log a disco rápido, reducir transacciones grandes)

📌 Esta es la query #1 para diagnosticar problemas de performance a nivel servidor
*/

In [ ]:
-- Query 2: Índices con fragmentación alta (> 30%)
SELECT 
    OBJECT_NAME(ips.object_id) AS tabla,
    i.name AS indice,
    ips.index_type_desc,
    ips.avg_fragmentation_in_percent AS fragmentacion_pct,
    ips.page_count AS paginas,
    CASE 
        WHEN ips.avg_fragmentation_in_percent > 30 THEN 'REBUILD'
        WHEN ips.avg_fragmentation_in_percent > 10 THEN 'REORGANIZE'
        ELSE 'OK'
    END AS accion_recomendada
FROM sys.dm_db_index_physical_stats(DB_ID(), NULL, NULL, NULL, 'SAMPLED') ips
INNER JOIN sys.indexes i ON ips.object_id = i.object_id AND ips.index_id = i.index_id
WHERE ips.page_count > 1000  -- Solo índices con más de 1000 páginas (8MB)
  AND ips.avg_fragmentation_in_percent > 10
ORDER BY ips.avg_fragmentation_in_percent DESC;

/*
🔧 Regla de oro:
- Fragmentación > 30%: ALTER INDEX ... REBUILD (online si Enterprise Edition)
- Fragmentación 10-30%: ALTER INDEX ... REORGANIZE (operación online, menos costosa)
- Fragmentación < 10%: no hacer nada

⚠️ Esta query puede tardar en bases grandes, ejecutar en horario de bajo tráfico
*/

In [ ]:
-- Query 1: Top 10 queries más costosas (por CPU total)
SELECT TOP 10
    qs.execution_count AS ejecuciones,
    qs.total_worker_time / 1000 AS cpu_total_ms,
    qs.total_worker_time / qs.execution_count / 1000 AS cpu_promedio_ms,
    qs.total_logical_reads AS reads_totales,
    SUBSTRING(qt.text, (qs.statement_start_offset/2)+1,
        ((CASE qs.statement_end_offset
            WHEN -1 THEN DATALENGTH(qt.text)
            ELSE qs.statement_end_offset
        END - qs.statement_start_offset)/2) + 1) AS query_text
FROM sys.dm_exec_query_stats qs
CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) qt
ORDER BY qs.total_worker_time DESC;

/*
📊 Interpretación:
- ejecuciones altas + cpu_promedio_ms alto = query problemática ejecutada frecuentemente
- ejecuciones bajas + cpu_total_ms alto = query ocasional muy costosa
- reads_totales altos = muchos I/O (revisar índices)

🎯 Acción: identifica queries con cpu_promedio > 1000ms y optimízalas
*/

## Estrategia de Monitoreo (Stack Completo)

### 1️⃣ **Nivel Query** (microsegundos a segundos)
- DMV: `sys.dm_exec_query_stats` → queries lentas
- Extended Events: capturar queries > 5s
- **Herramienta**: Query Store (SQL Server 2016+)

### 2️⃣ **Nivel Índices** (diario/semanal)
- DMV: `sys.dm_db_index_physical_stats` → fragmentación
- DMV: `sys.dm_db_index_usage_stats` → índices nunca usados
- **Job SQL Agent**: rebuild/reorganize índices nocturnos

### 3️⃣ **Nivel Servidor** (continuamente)
- DMV: `sys.dm_os_wait_stats` → cuellos de botella globales
- DMV: `sys.dm_os_performance_counters` → CPU, memoria, disco
- **Herramienta**: Azure Monitor, Grafana + Prometheus

### 4️⃣ **Nivel Aplicación** (end-to-end)
- Application Insights (Azure)
- Logging estructurado (Serilog, NLog)
- Distributed tracing (Jaeger, Zipkin)

## 🟢 Ejercicio Básico
Ejecuta la query de "Top 10 costosas" y documenta:
1. ¿Cuál es la query con mayor CPU promedio?
2. ¿Cuántas veces se ejecutó en el período capturado?
3. ¿Qué harías para optimizarla? (hipótesis sin ver el plan aún)

## 🟠 Ejercicio Intermedio
Crea un dashboard de monitoreo SQL con estas 4 métricas en vista:
1. Queries > 5s (últimas 24h)
2. Índices con fragmentación > 30%
3. Top 5 wait types
4. Espacio en disco disponible (usar `sys.dm_os_volume_stats`)

**Entregable**: 4 consultas SQL listas para ejecutar cada hora

## 🔴 Ejercicio Avanzado
Diseña una estrategia de observabilidad completa:
1. **Alertas** (¿qué condiciones disparan notificación inmediata?)
2. **Baseline** (¿qué métricas capturar diario para tendencias?)
3. **Runbook** (procedimiento cuando llega alerta de "query lenta")
4. **Retención** (¿cuánto tiempo guardar métricas históricas?)

---

## Errores Comunes

❌ **Solo monitorear en crisis**: métricas históricas permiten comparar normal vs anómalo
❌ **Alertas por todo**: si todo es urgente, nada es urgente (alert fatigue)
❌ **No automatizar**: checking manual diario no escala
❌ **Ignorar wait stats**: son la brújula para diagnosticar a nivel servidor
❌ **No tener SLA definido**: ¿qué es "lento"? (define: p95 < 2s, p99 < 5s)

**Siguiente:** `10_proyecto_senior.ipynb` → proyecto integrador capstone

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](08_almacenamiento_y_mantenimiento.ipynb) | [Siguiente ➡️](10_proyecto_senior.ipynb)
---
